In [ ]:
# Import necessary libraries
import pandas as pd
import json
import plotly.express as px

In [ ]:
# Load columns_info.json
with open('../data/columns_info.json') as f:
    columns_info = json.load(f)

# Load chiller_plant_dataset.parquet
chiller_data = pd.read_parquet('../data/chiller_plant_dataset.parquet')

In [ ]:
chiller_data_raw = pd.read_parquet('../data/chiller_plant_dataset.parquet')

## Preprocessing

In [ ]:
chiller_data.info()

In [ ]:
chiller_data.describe()

In [ ]:
chiller_data.head()

In [ ]:
chiller_data.isnull().sum().sum()

In [ ]:
chiller_data.columns

### Drop columns

In [ ]:
import ast

column_list = chiller_data.columns.to_list()
columns_info_list = []
for key in columns_info:
    tup = ast.literal_eval(key)
    columns_info_list.append(tup)
columns_to_drop = [col for col in column_list if col not in columns_info_list]
columns_to_drop.append(('chiller_6', 'power'))
chiller_data = chiller_data.drop(columns=columns_to_drop)
chiller_data.shape

### Cleaning missing value

In [ ]:
chiller_data.ffill(inplace=True)
chiller_data.bfill(inplace=True)

In [ ]:
chiller_data.isnull().sum().sum()

### Check negative value

In [ ]:
# Check which columns have negative values
negative_columns = chiller_data.columns[chiller_data.lt(0).any()]

# Count negative values in each column
negative_counts = chiller_data[negative_columns].lt(0).sum().sum()

print("negative values:")
print(negative_counts)

In [ ]:
# Replace negative values with NaN
chiller_data[negative_columns] = chiller_data[negative_columns].mask(chiller_data[negative_columns] < 0)
# Forward fill NaN values
chiller_data.ffill(inplace=True)
chiller_data.bfill(inplace=True)

# Verify if negative values are replaced
negative_counts_after_fill = chiller_data[negative_columns].lt(0).sum().sum()

print("negative values after forward filling:")
print(negative_counts_after_fill)

## EDA

### Correlation Analysis

target = power consumption of chiller/pump/cooling tower 

feature = others variable

In [ ]:
power_columns = [col for col in chiller_data.columns if 'power' in col[1]]
chiller_data[power_columns].describe()

#### Correlation between features

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

is_device = ['chiller', 'cdp', 'chp', 'ct']
feature_columns = [col for col in chiller_data.columns if col[0] == 'chiller_1' and col[1] != 'power']
feature_columns = [col for col in chiller_data.columns 
                   if (col[0].split('_')[0] not in is_device and 'power' not in col[1]) 
                   or col in feature_columns]

# Check the number of selected feature columns
num_features = len(feature_columns)
print(f"Number of feature columns: {num_features}")

# Extract the feature data
features_data = chiller_data[feature_columns]

# Calculate the Spearman correlation matrix
spearman_corr_matrix = features_data.corr(method='spearman')

# Display the Spearman correlation matrix
print(spearman_corr_matrix)

# Set up the matplotlib figure
plt.figure(figsize=(15, 15))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(spearman_corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, linewidths=.5, cbar_kws={"shrink": .5})

# Set the title
plt.title('Spearman Correlation Matrix', size=16)

# Show the plot
plt.show()

# Find pairs with absolute correlation greater than 0.7
threshold = 0.7
high_corr_pairs = []

for i in range(len(spearman_corr_matrix.columns)):
    for j in range(i + 1, len(spearman_corr_matrix.columns)):
        corr_value = spearman_corr_matrix.iloc[i, j]
        if abs(corr_value) > threshold:
            feature_pair = (spearman_corr_matrix.columns[i], spearman_corr_matrix.columns[j], corr_value)
            high_corr_pairs.append(feature_pair)

# Convert to DataFrame for better readability
high_corr_pairs_df = pd.DataFrame(high_corr_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])

print(high_corr_pairs_df)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

is_device = ['chiller', 'cdp', 'chp', 'ct']
feature_columns = [col for col in chiller_data.columns 
                   if col[0] == 'chiller_1' and col[1] != 'power']

# Check the number of selected feature columns
num_features = len(feature_columns)
print(f"Number of feature columns: {num_features}")

# Extract the feature data
features_data = chiller_data[feature_columns]

# Calculate the Spearman correlation matrix
spearman_corr_matrix = features_data.corr(method='spearman')

# Display the Spearman correlation matrix
print(spearman_corr_matrix)

# Set up the matplotlib figure
plt.figure(figsize=(15, 15))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(spearman_corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, linewidths=.5, cbar_kws={"shrink": .5})

# Set the title
plt.title('Spearman Correlation Matrix', size=16)

# Show the plot
plt.show()

#### Correlation to power

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Define the list of power columns
power_columns = [col for col in chiller_data.columns if 'chiller' in col[0] and 'power' in col[1]]

is_device = ['chiller', 'cdp', 'chp', 'ct']
drop_properties = ['status_read', 'mode', 'current', 'setpoint_read']

# Calculate and plot top 20 Spearman correlations for each power column
for power_col in power_columns:
    compare_columns = [ccol for ccol in chiller_data.columns if (ccol[0] == power_col[0] or ccol[0].split('_')[0] not in is_device)
                       and 'power' not in ccol[1]]
    compare_columns = [ccol for ccol in compare_columns if ccol[1] not in drop_properties]
    
    X = chiller_data[compare_columns]
    y = chiller_data[power_col]
    
    # Calculate Spearman correlation
    correlations = X.apply(lambda col: col.corr(y, method='spearman'))
    corr_series = correlations.sort_values(ascending=False)
    corr_series.index = ['_'.join(col) if isinstance(col, tuple) else col for col in corr_series.index]
    
    plt.figure(figsize=(10, 12))
    sns.barplot(x=corr_series.values, y=corr_series.index)
    plt.title(f'Top 20 Spearman Correlations with {power_col}')
    plt.xlabel('Spearman Correlation')
    plt.ylabel('Variables')
    plt.show()

#### Corr Chiller_1

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Define the list of power columns
power_columns = [col for col in chiller_data.columns if 'chiller' in col[0] and 'power' in col[1]]


# Calculate and plot top 20 Spearman correlations for each power column
for power_col in power_columns:
    compare_columns = [ccol for ccol in chiller_data.columns if ccol[0]==power_col[0]
                       and 'power' not in ccol[1]]
    
    X = chiller_data[compare_columns]
    y = chiller_data[power_col]
    
    # Calculate Spearman correlation
    correlations = X.apply(lambda col: col.corr(y, method='spearman'))
    corr_series = correlations.sort_values(ascending=False)
    corr_series.index = ['_'.join(col) if isinstance(col, tuple) else col for col in corr_series.index]
    
    plt.figure(figsize=(10, 12))
    sns.barplot(x=corr_series.values, y=corr_series.index)
    plt.title(f'Top 20 Spearman Correlations with {power_col}')
    plt.xlabel('Spearman Correlation')
    plt.ylabel('Variables')
    plt.show()

#### Mutual Information

In [ ]:
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Define the list of power columns
power_columns = [col for col in chiller_data.columns if 'chiller' in col[0] and 'power' in col[1]]

is_device = ['chiller','cdp','chp','ct']
drop_properties = ['status_read','mode','current','setpoint_read']

# Calculate and plot top 20 mutual information for each power column
for power_col in power_columns:
    compare_columns = [ccol for ccol in chiller_data.columns if (ccol[0]==power_col[0] or ccol[0].split('_')[0] not in is_device)
                       and 'power' not in ccol[1]]
    compare_columns = [ccol for ccol in compare_columns if ccol[1] not in drop_properties]
    X = chiller_data[compare_columns]
    y = chiller_data[power_col]
    
    mi = mutual_info_regression(X, y)
    mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
    mi_series.index = ['_'.join(col) if isinstance(col, tuple) else col for col in mi_series.index]
    
    plt.figure(figsize=(10, 12))
    sns.barplot(x=mi_series.values, y=mi_series.index)
    plt.title(f'Top 20 Mutual Information with {power_col}')
    plt.xlabel('Mutual Information')
    plt.ylabel('Variables')
    plt.show()

#### Power plot

In [ ]:
fig = px.line(x=chiller_data.index, y=chiller_data['chiller_1','power'])
fig.show()

In [ ]:
fig = px.line(x=chiller_data.index, y=chiller_data['chiller_2','power'])
fig.show()

In [ ]:
fig = px.line(x=chiller_data.index, y=chiller_data['chiller_3','power'])
fig.show()

In [ ]:
fig = px.line(x=chiller_data.index, y=chiller_data['chiller_4','power'])
fig.show()

In [ ]:
fig = px.line(x=chiller_data.index, y=chiller_data['chiller_5','power'])
fig.show()

In [ ]:
fig = px.line(x=chiller_data.index, y=chiller_data['plant','power'])
fig.show()

In [ ]:
fig = px.line(x=chiller_data.index, y=chiller_data['plant','power_all_chillers'])
fig.show()

In [ ]:
fig = px.line(x=chiller_data.index, y=chiller_data['plant','power_all_cts'])
fig.show()

In [ ]:
fig = px.line(x=chiller_data.index, y=chiller_data['plant','power_all_cdps'])
fig.show()

In [ ]:
fig = px.line(x=chiller_data.index, y=chiller_data['plant','power_all_chps'])
fig.show()

#### Corr plant power

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Define the list of power columns
power_columns = [col for col in chiller_data.columns if 'plant' in col[0] and 'power' in col[1]]

is_device = ['chiller', 'cdp', 'chp', 'ct']
drop_properties = ['status_read', 'mode', 'current', 'setpoint_read']

# Calculate and plot top 20 Spearman correlations for each power column
for power_col in power_columns:
    compare_columns = [ccol for ccol in chiller_data.columns if (ccol[0] == power_col[0] or ccol[0].split('_')[0] not in is_device)
                       and 'power' not in ccol[1]]
    compare_columns = [ccol for ccol in compare_columns if ccol[1] not in drop_properties]
    
    X = chiller_data[compare_columns]
    y = chiller_data[power_col]
    
    # Calculate Spearman correlation
    correlations = X.apply(lambda col: col.corr(y, method='spearman'))
    corr_series = correlations.sort_values(ascending=False)
    corr_series.index = ['_'.join(col) if isinstance(col, tuple) else col for col in corr_series.index]
    
    plt.figure(figsize=(10, 12))
    sns.barplot(x=corr_series.values, y=corr_series.index)
    plt.title(f'Top 20 Spearman Correlations with {power_col}')
    plt.xlabel('Spearman Correlation')
    plt.ylabel('Variables')
    plt.show()

#### Mutual Info Plant

In [ ]:
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Define the list of power columns
power_columns = [col for col in chiller_data.columns if 'plant' in col[0] and 'power' in col[1]]

is_device = ['chiller', 'cdp', 'chp', 'ct']
drop_properties = ['status_read', 'mode', 'current', 'setpoint_read']

# Calculate and plot top 20 Spearman correlations for each power column
for power_col in power_columns:
    compare_columns = [ccol for ccol in chiller_data.columns if (ccol[0] == power_col[0] or ccol[0].split('_')[0] not in is_device)
                       and 'power' not in ccol[1]]
    compare_columns = [ccol for ccol in compare_columns if ccol[1] not in drop_properties]
    
    X = chiller_data[compare_columns]
    y = chiller_data[power_col]
    
    mi = mutual_info_regression(X, y)
    mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
    mi_series.index = ['_'.join(col) if isinstance(col, tuple) else col for col in mi_series.index]
    
    plt.figure(figsize=(10, 12))
    sns.barplot(x=mi_series.values, y=mi_series.index)
    plt.title(f'Top 20 Mutual Information with {power_col}')
    plt.xlabel('Mutual Information')
    plt.ylabel('Variables')
    plt.show()

## Feature Engineering

select proper feature from EDA

#### Drop unneccesary columns

For chilller:

    Target: power

    Feature to keep

        cooling_rate ?

        cond_leaving_water_temperature

        cond_entering_water_temperature

        evap_leaving_water_temperature

        evap_entering_water_temperature

        cond_water_flow_rate

        evap_water_flow_rate


In [ ]:
chiller_data.describe()

## System Identification

In [ ]:
chiller_data_fit = chiller_data.copy()
chiller_data_fit.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in chiller_data_fit.columns]

#### Chiller_1

##### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# ch1_feature = [col for col in chiller_data_fit.columns if 'chiller_1' in col]

ch1_feature = ['chiller_1_evap_leaving_water_temperature',
               'chiller_1_evap_entering_water_temperature',
               'chiller_1_cond_leaving_water_temperature',
               'chiller_1_cond_entering_water_temperature',
               'chiller_1_evap_water_flow_rate',
               'chiller_1_cond_water_flow_rate']

ch1_target = ['chiller_1_power']

X = chiller_data_fit[ch1_feature]
y = chiller_data_fit[ch1_target]

X_train, y_train = X[:int(0.6*len(X))], y[:int(0.6*len(y))]
X_test, y_test = X[int(0.6*len(X)):], y[int(0.6*len(y)):]

# Train the model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predict
y_pred_lr = lr_model.predict(X_test)

# Performance
mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f"Linear Regression MSE: {mse_lr}")

In [ ]:
y_pred_lr_df = pd.DataFrame(y_pred_lr, index=y_test.index, columns=y_test.columns)

In [ ]:
fig = px.line(x=X_test.index, y=y_pred_lr_df['chiller_1_power'])
fig.update_traces(line_color='red', name='Chiller 1 Power')
fig.add_trace(px.line(x=X_test.index, y=y_test['chiller_1_power']).data[0])
fig.show()

##### Clipped Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# ch1_feature = [col for col in chiller_data_fit.columns if 'chiller_1' in col]

ch1_feature = ['chiller_1_evap_leaving_water_temperature',
               'chiller_1_evap_entering_water_temperature',
               'chiller_1_cond_leaving_water_temperature',
               'chiller_1_cond_entering_water_temperature',
               'chiller_1_evap_water_flow_rate',
               'chiller_1_cond_water_flow_rate']

ch1_target = ['chiller_1_power']

X = chiller_data_fit[ch1_feature]
y = chiller_data_fit[ch1_target]

X_train, y_train = X[:int(0.6*len(X))], y[:int(0.6*len(y))]
X_test, y_test = X[int(0.6*len(X)):], y[int(0.6*len(y)):]

# Train the model
ridge_model = Ridge(alpha=1)
ridge_model.fit(X_train, y_train)

# Predict
y_pred_ridge = ridge_model.predict(X_test)

# Performance
mse_lr = mean_squared_error(y_test, y_pred_ridge)
print(f"Linear Regression MSE: {mse_lr}")

y_pred_ridge_clipped = y_pred_ridge.clip(min=0)
# Performance after clipping
mse_ridge_clipped = mean_squared_error(y_test, y_pred_ridge_clipped)
print(f"Ridge Regression MSE after Clipping: {mse_ridge_clipped}")

In [ ]:
y_pred_ridge_df = pd.DataFrame(y_pred_ridge, index=y_test.index, columns=y_test.columns)
y_pred_ridge_clipped_df = pd.DataFrame(y_pred_ridge_clipped, index=y_test.index, columns=y_test.columns)
fig = px.line(x=X_test.index, y=y_pred_ridge_clipped_df['chiller_1_power'])
fig.update_traces(line_color='red', name='Chiller 1 Power')
fig.add_trace(px.line(x=X_test.index, y=y_test['chiller_1_power']).data[0])
fig.show()

##### Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

ch1_feature = ['chiller_1_evap_leaving_water_temperature',
               'chiller_1_evap_entering_water_temperature',
               'chiller_1_cond_leaving_water_temperature',
               'chiller_1_cond_entering_water_temperature',
               'chiller_1_evap_water_flow_rate',
               'chiller_1_cond_water_flow_rate']

ch1_target = ['chiller_1_power']

X = chiller_data_fit[ch1_feature]
y = chiller_data_fit[ch1_target]

X_train, y_train = X[:int(0.6*len(X))], y[:int(0.6*len(y))]
X_test, y_test = X[int(0.6*len(X)):], y[int(0.6*len(y)):]
# Random Forest Model
rf_model = RandomForestRegressor(n_estimators=400,
                           max_depth=30,
                           min_samples_split=10,
                           min_samples_leaf=6, 
                           random_state=42, 
                           bootstrap=True)
rf_model.fit(X_train, y_train.values.ravel())

y_pred_rf = rf_model.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest MSE: {mse_rf}")

# Clipping negative predictions to zero
y_pred_rf_clipped = y_pred_rf.clip(min=0)

# Performance after clipping
mse_rf_clipped = mean_squared_error(y_test, y_pred_rf_clipped)
print(f"Random Forest MSE after Clipping: {mse_rf_clipped}")

In [ ]:
y_pred_rf_clipped_df = pd.DataFrame(y_pred_rf_clipped, index=y_test.index, columns=y_test.columns)
fig = px.line(x=X_test.index, y=y_pred_rf_clipped_df['chiller_1_power'])
fig.update_traces(line_color='red', name='Chiller 1 Power')
fig.add_trace(px.line(x=X_test.index, y=y_test['chiller_1_power']).data[0])

##### Grid Search Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'min_samples_split': [3, 5, 10, 12],
    'min_samples_leaf': [2, 4, 6, 10],
}

# Create a base model
rf = RandomForestRegressor(n_estimators=400,
                           max_depth=30, 
                           random_state=42, 
                           bootstrap=True)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                          cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train, y_train.values.ravel())

# Print the best parameters and the best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

# Use the best model
best_rf_model = grid_search.best_estimator_

# Predict
y_pred_best_rf = best_rf_model.predict(X_test)

# Performance
mse_best_rf = mean_squared_error(y_test, y_pred_best_rf)
print(f"Random Forest MSE (Best Model): {mse_best_rf}")

# Clipping negative predictions to zero
y_pred_best_rf_clipped = y_pred_best_rf.clip(min=0)

# Performance after clipping
mse_best_rf_clipped = mean_squared_error(y_test, y_pred_best_rf_clipped)
print(f"Random Forest MSE after Clipping (Best Model): {mse_best_rf_clipped}")

In [ ]:
y_pred_best_rf_clipped_df = pd.DataFrame(y_pred_best_rf_clipped, index=y_test.index, columns=y_test.columns)
fig = px.line(x=X_test.index, y=y_pred_best_rf_clipped_df['chiller_1_power'])
fig.update_traces(line_color='red', name='Chiller 1 Power')
fig.add_trace(px.line(x=X_test.index, y=y_test['chiller_1_power']).data[0])

##### ARIMA Regression with predictor variables

##### SARIMAX Regression

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import numpy as np

ch1_feature = [
    'chiller_1_evap_leaving_water_temperature',
    'chiller_1_evap_entering_water_temperature',
    'chiller_1_cond_leaving_water_temperature',
    'chiller_1_cond_entering_water_temperature',
    'chiller_1_evap_water_flow_rate',
    'chiller_1_cond_water_flow_rate'
]

ch1_target = ['chiller_1_power']

X = chiller_data_fit[ch1_feature]
y = chiller_data_fit[ch1_target]

# Ensure the index has a datetime frequency
X.index = pd.to_datetime(X.index)
y.index = pd.to_datetime(y.index)
X.index.freq = pd.infer_freq(X.index)
y.index.freq = pd.infer_freq(y.index)

# Splitting the data into train and test sets
train_size = int(0.6 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Define the SARIMAX model
model = SARIMAX(endog=y_train, exog=X_train, order=(2, 1, 3), seasonal_order=(0,0,0,0))

# Fit the SARIMAX model
fitted_model = model.fit()

# Print the coefficients
print("Coefficients of the SARIMAX model:")
print(fitted_model.params)

# Print the summary of the fitted model
print(fitted_model.summary())


# Forecast using the SARIMAX model
forecast = fitted_model.forecast(steps=len(X_test), exog=X_test)
#forecast = np.maximum(forecast, 0)

# Calculate MSE
mse_sarimax = mean_squared_error(y_test, forecast)
print(f"SARIMAX MSE: {mse_sarimax}")

In [ ]:
forecast_df = pd.DataFrame(forecast.values, index=y_test.index, columns=y_test.columns)
fig = px.line(x=X_test.index, y=forecast_df['chiller_1_power'])
fig.update_traces(line_color='red', name='Chiller 1 Power')
fig.add_trace(px.line(x=X_test.index, y=y_test['chiller_1_power']).data[0])

##### SARIMAX with log

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import numpy as np

# Define features and target
ch1_feature = [
    'chiller_1_evap_leaving_water_temperature',
    'chiller_1_evap_entering_water_temperature',
    'chiller_1_cond_leaving_water_temperature',
    'chiller_1_cond_entering_water_temperature',
    'chiller_1_evap_water_flow_rate',
    'chiller_1_cond_water_flow_rate'
]

ch1_target = ['chiller_1_power']

X = chiller_data_fit[ch1_feature]
y = chiller_data_fit[ch1_target]

# Apply log transformation to the target variable to ensure non-negative predictions
y_log = np.log1p(y)  # log1p is used to avoid log(0) issues

# Splitting the data into train and test sets
train_size = int(0.6 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train_log, y_test_log = y_log[:train_size], y_log[train_size:]

# Define the SARIMAX model
model = SARIMAX(endog=y_train_log, exog=X_train, order=(1, 1, 1), seasonal_order=(0, 0, 0, 0))

# Fit the SARIMAX model
fitted_model = model.fit()

# Forecast using the SARIMAX model
forecast_log = fitted_model.forecast(steps=len(X_test), exog=X_test)

# Transform the predictions back to the original scale
forecast = np.expm1(forecast_log)  # expm1 is the inverse of log1p

# Calculate MSE
mse_sarimax = mean_squared_error(y_test, forecast)
print(f"SARIMAX MSE: {mse_sarimax}")

# Ensure predictions are non-negative
# forecast = np.maximum(forecast, 0)
mse_sarimax_clipped = mean_squared_error(y_test, forecast)
print(f"SARIMAX MSE after clipping: {mse_sarimax_clipped}")

In [ ]:
forecast_df = pd.DataFrame(forecast.values, index=y_test.index, columns=y_test.columns)
fig = px.line(x=X_test.index, y=forecast_df['chiller_1_power'])
fig.update_traces(line_color='red', name='Chiller 1 Power')
fig.add_trace(px.line(x=X_test.index, y=y_test['chiller_1_power']).data[0])

##### SARIMAX feature engineering

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
import numpy as np

ch1_feature = [
    'chiller_1_evap_leaving_water_temperature',
    'chiller_1_evap_entering_water_temperature',
    'chiller_1_cond_leaving_water_temperature',
    'chiller_1_cond_entering_water_temperature',
    'chiller_1_evap_water_flow_rate',
    'chiller_1_cond_water_flow_rate',
    'chiller_1_demand_limit_setpoint_read'
]

ch1_target = ['chiller_1_power']

X = chiller_data_fit[ch1_feature]
y = chiller_data_fit[ch1_target]

# Ensure the index has a datetime frequency
X.index = pd.to_datetime(X.index)
y.index = pd.to_datetime(y.index)
X.index.freq = pd.infer_freq(X.index)
y.index.freq = pd.infer_freq(y.index)

# Splitting the data into train and test sets
train_size = int(0.6 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

decomposition = seasonal_decompose(y, period=1)  # Adjust period based on your data's seasonal cycle
# Plot seasonal components
decomposition.plot()
plt.show()

# ACF and PACF plots
plot_acf(y)
plot_pacf(y)
plt.show()

In [ ]:
# Define the SARIMAX model
model = SARIMAX(endog=y_train, exog=X_train, order=(2, 1, 3), seasonal_order=(0,0,0,0))

# Fit the SARIMAX model
fitted_model = model.fit()

# Print the coefficients
print("Coefficients of the SARIMAX model:")
print(fitted_model.params)

# Print the summary of the fitted model
print(fitted_model.summary())


# Forecast using the SARIMAX model
forecast = fitted_model.forecast(steps=len(X_test), exog=X_test)
#forecast = np.maximum(forecast, 0)

# Calculate MSE
mse_sarimax = mean_squared_error(y_test, forecast)
print(f"SARIMAX MSE: {mse_sarimax}")

In [ ]:
forecast = np.maximum(forecast, 0)

# Calculate MSE
mse_sarimax = mean_squared_error(y_test, forecast)
print(f"SARIMAX MSE: {mse_sarimax}")

In [ ]:
forecast_df = pd.DataFrame(forecast.values, index=y_test.index, columns=y_test.columns)
fig = px.line(x=X_test.index, y=forecast_df['chiller_1_power'])
fig.update_traces(line_color='red', name='Chiller 1 Power')
fig.add_trace(px.line(x=X_test.index, y=y_test['chiller_1_power']).data[0])

## System Identification Plant

##### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

plant_features = {'plant_power':['condenser_water_loop_flow_rate',
                                 'chilled_water_loop_flow_rate',
                                 'condenser_water_loop_water_delta_temperature',
                                 'chilled_water_loop_water_delta_temperature',
                                 'plant_number_of_running_chps'],
                    'plant_power_all_cdps':['plant_avg_cdp_speed',
                                          'condenser_water_loop_flow_rate'],
                    'plant_power_all_chps':['plant_avg_chp_speed',
                                          'chilled_water_loop_flow_rate'],
                    'plant_power_all_chillers':['condenser_water_loop_flow_rate',
                                 'chilled_water_loop_flow_rate',
                                 'condenser_water_loop_water_delta_temperature',
                                 'chilled_water_loop_water_delta_temperature',],
                    'plant_power_all_cts':['condenser_water_loop_flow_rate',
                                           'plant_number_of_running_cts',
                                           'plant_avg_ct_speed',
                                           'condenser_water_loop_water_delta_temperature']}

plant_targets = [[col] for col in chiller_data_fit.columns if 'plant' in col and 'power' in col]

for target in plant_targets:
    X = chiller_data_fit[plant_features[target[0]]]
    y = chiller_data_fit[target]

    X_train, y_train = X[:int(0.6*len(X))], y[:int(0.6*len(y))]
    X_test, y_test = X[int(0.6*len(X)):], y[int(0.6*len(y)):]
    # Random Forest Model
    rf_model = RandomForestRegressor(n_estimators=400,
                           max_depth=30,
                           min_samples_split=10,
                           min_samples_leaf=6, 
                           random_state=42, 
                           bootstrap=True)
    rf_model.fit(X_train, y_train.values.ravel())

    y_pred_rf = rf_model.predict(X_test)

    mse_rf = mean_squared_error(y_test, y_pred_rf)
    print(f"Random Forest MSE: {mse_rf}")

    # Clipping negative predictions to zero
    y_pred_rf_clipped = y_pred_rf.clip(min=0)

    # Performance after clipping
    mse_rf_clipped = mean_squared_error(y_test, y_pred_rf_clipped)
    print(f"Random Forest MSE after Clipping: {mse_rf_clipped}")
    y_pred_rf_clipped_df = pd.DataFrame(y_pred_rf_clipped, index=y_test.index, columns=y_test.columns)
    fig = px.line(x=X_test.index, y=y_pred_rf_clipped_df[target[0]], title=f'{target}')
    fig.update_traces(line_color='red', name=f'{target}')
    fig.add_trace(px.line(x=X_test.index, y=y_test[target[0]]).data[0])
    fig.show()

##### NARX

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import plotly.express as px

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the function to create lagged features
def create_lagged_features(df, target_col, lag, exog_cols):
    df_lagged = df.copy()
    for l in range(1, lag + 1):
        df_lagged[f'{target_col}_lag{l}'] = df[target_col].shift(l)
        for exog_col in exog_cols:
            df_lagged[f'{exog_col}_lag{l}'] = df[exog_col].shift(l)
    df_lagged = df_lagged.dropna()
    return df_lagged

# Parameters
lag = 3  # Number of lags to use

plant_features = {'plant_power': ['condenser_water_loop_flow_rate',
                                  'chilled_water_loop_flow_rate',
                                  'condenser_water_loop_water_delta_temperature',
                                  'chilled_water_loop_water_delta_temperature',
                                  'plant_number_of_running_chps'],
                  'plant_power_all_cdps': ['plant_avg_cdp_speed',
                                           'condenser_water_loop_flow_rate'],
                  'plant_power_all_chps': ['plant_avg_chp_speed',
                                           'chilled_water_loop_flow_rate'],
                  'plant_power_all_chillers': ['condenser_water_loop_flow_rate',
                                               'chilled_water_loop_flow_rate',
                                               'condenser_water_loop_water_delta_temperature',
                                               'chilled_water_loop_water_delta_temperature'],
                  'plant_power_all_cts': ['condenser_water_loop_flow_rate',
                                          'plant_number_of_running_cts',
                                          'plant_avg_ct_speed',
                                          'condenser_water_loop_water_delta_temperature']}

plant_targets = [[col] for col in chiller_data_fit.columns if 'plant' in col and 'power' in col]

class NARXModel(nn.Module):
    def __init__(self, input_dim):
        super(NARXModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, 50, batch_first=True)
        self.fc = nn.Linear(50, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

for target in plant_targets:
    target_col = target[0]
    exog_cols = plant_features[target_col]

    use_cols = [col for col in chiller_data_fit.columns if col==target_col or col in exog_cols]
    df_use = chiller_data_fit[use_cols]

    # Create lagged features
    df_use_lagged = create_lagged_features(df_use, target_col, lag, exog_cols)
    X = df_use_lagged.drop(columns=[target_col])
    y = df_use_lagged[target]

    # Splitting the data into train and test sets
    train_size = int(0.6 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Normalize the features
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
    y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

    # Calculate the correct number of features
    num_features = X_train.shape[1] // (lag + 1)

    print(f"X_train shape: {X_train.shape}")
    print(f"X_train_scaled shape before reshape: {X_train_scaled.shape}")
    print(f"Expected reshape dimensions: ({X_train_scaled.shape[0]}, {lag + 1}, {num_features})")

    # Ensure the number of elements is divisible by the new shape
    if X_train_scaled.size % (lag + 1 * num_features) != 0:
        raise ValueError(f"Cannot reshape array of size {X_train_scaled.size} into shape ({X_train_scaled.shape[0]}, {lag + 1}, {num_features})")

    # Reshape for LSTM
    X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], lag + 1, num_features))
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], lag + 1, num_features))

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32).to(device)

    # Define the model
    model = NARXModel(input_dim=X_train_tensor.shape[2]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training the model
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Make predictions
    model.eval()
    with torch.no_grad():
        y_pred_scaled = model(X_test_tensor).cpu().numpy()
        y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # Clipping negative predictions to zero
    y_pred_clipped = y_pred.clip(min=0)

    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    mse_clipped = mean_squared_error(y_test, y_pred_clipped)
    print(f"NARX MSE for {target_col}: {mse}")
    print(f"NARX MSE after Clipping for {target_col}: {mse_clipped}")

    # Visualize the results
    y_pred_clipped_df = pd.DataFrame(y_pred_clipped, index=y_test.index, columns=[target_col])
    fig = px.line(x=X_test.index, y=y_pred_clipped_df[target_col], title=f'{target_col}')
    fig.update_traces(line_color='red', name=f'{target_col}')
    fig.add_trace(px.line(x=X_test.index, y=y_test[target_col]).data[0])
    fig.show()


In [ ]:

import torch

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
 
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {torch.cuda.current_device()}")
       
print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import plotly.express as px

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the function to create lagged features
def create_lagged_features(df, target_col, lag, exog_cols):
    df_lagged = df.copy()
    for l in range(1, lag + 1):
        df_lagged[f'{target_col}_lag{l}'] = df[target_col].shift(l)
        for exog_col in exog_cols:
            df_lagged[f'{exog_col}_lag{l}'] = df[exog_col].shift(l)
    df_lagged = df_lagged.dropna()
    return df_lagged

# Parameters
lag = 3  # Number of lags to use

plant_features = {'plant_power': ['condenser_water_loop_flow_rate',
                                  'chilled_water_loop_flow_rate',
                                  'condenser_water_loop_water_delta_temperature',
                                  'chilled_water_loop_water_delta_temperature',
                                  'plant_number_of_running_chps'],
                  'plant_power_all_cdps': ['plant_avg_cdp_speed',
                                           'condenser_water_loop_flow_rate'],
                  'plant_power_all_chps': ['plant_avg_chp_speed',
                                           'chilled_water_loop_flow_rate'],
                  'plant_power_all_chillers': ['condenser_water_loop_flow_rate',
                                               'chilled_water_loop_flow_rate',
                                               'condenser_water_loop_water_delta_temperature',
                                               'chilled_water_loop_water_delta_temperature'],
                  'plant_power_all_cts': ['condenser_water_loop_flow_rate',
                                          'plant_number_of_running_cts',
                                          'plant_avg_ct_speed',
                                          'condenser_water_loop_water_delta_temperature']}

plant_targets = [[col] for col in chiller_data_fit.columns if 'plant' in col and 'power' in col]

class NARXModel(nn.Module):
    def __init__(self, input_dim):
        super(NARXModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

for target in plant_targets:
    target_col = target[0]
    exog_cols = plant_features[target_col]

    use_cols = [col for col in chiller_data_fit.columns if col==target_col or col in exog_cols]
    df_use = chiller_data_fit[use_cols]

    # Create lagged features
    df_use_lagged = create_lagged_features(df_use, target_col, lag, exog_cols)
    X = df_use_lagged.drop(columns=[target_col])
    y = df_use_lagged[target]

    # Splitting the data into train and test sets
    train_size = int(0.6 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Normalize the features
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
    y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32).to(device)

    # Define the model
    model = NARXModel(input_dim=X_train_tensor.shape[1]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training the model
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Make predictions
    model.eval()
    with torch.no_grad():
        y_pred_scaled = model(X_test_tensor).cpu().numpy()
        y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # Clipping negative predictions to zero
    y_pred_clipped = y_pred.clip(min=0)

    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    mse_clipped = mean_squared_error(y_test, y_pred_clipped)
    print(f"NARX MSE for {target_col}: {mse}")
    print(f"NARX MSE after Clipping for {target_col}: {mse_clipped}")

    # Visualize the results
    y_pred_clipped_df = pd.DataFrame(y_pred_clipped, index=y_test.index, columns=[target_col])
    fig = px.line(x=X_test.index, y=y_pred_clipped_df[target_col], title=f'{target_col}')
    fig.update_traces(line_color='red', name=f'{target_col}')
    fig.add_trace(px.line(x=X_test.index, y=y_test[target_col]).data[0])
    fig.show()
